### Population Growth and PCP Density per State (2020)

***NOTE***: 

- Datset Aquired from PolicyMap / Census: Decennial Census and American Community Survey (ACS)

- Goal is to create a map visual in PowerBI using this dataset to display the relationship between population growth and pcp density per state in the year 2020.

In [ ]:
"""
Create a state-level merged CSV with:
- PCP counts aggregated to state for 2020
- 2020 population per state
- PCP_per_100k_2020 computed as (PCP_2020 / Population_2020) * 100000

Inputs:
- primary_care_physicians_2018_2021_cleaned.csv
- us_censusbureau_population_2020.csv

Output:
- pcp_population_2020_merged.csv
"""

import pandas as pd
import os
import sys

# ---------- CONFIG: adjust file paths if your files are elsewhere ----------
PCP_CSV = "Data_Files_Cleaned/primary_care_physicians_2018_2021_cleaned.csv"
POP_CSV = "Data_Files_Cleaned/us_censusbureau_population_2020.csv"
OUTPUT_CSV = "Data_Files_Cleaned/pcp_population_2020_merged.csv"
# -------------------------------------------------------------------------

def load_csv(path):
    if not os.path.exists(path):
        print(f"ERROR: File not found: {path}")
        sys.exit(1)
    return pd.read_csv(path, dtype=str)  # read as str to clean/inspect before numeric conversion

def main():
    # Load files
    pcp_raw = load_csv(PCP_CSV)
    pop_raw = load_csv(POP_CSV)

    # Normalize column names (strip whitespace)
    pcp_raw.columns = [c.strip() for c in pcp_raw.columns]
    pop_raw.columns = [c.strip() for c in pop_raw.columns]

    # ---------- Identify expected columns ----------
    # PCP: expect county-level with State abbrev in column named 'State' or 'state'
    state_col_pcp = None
    for cand in ['State', 'state', 'STATE']:
        if cand in pcp_raw.columns:
            state_col_pcp = cand
            break
    if state_col_pcp is None:
        raise Exception("Could not find a 'State' column in PCP file. Columns: " + ", ".join(pcp_raw.columns))

    # Look for Primary_Care_Physicians_2020 column
    pcp_2020_col = None
    for c in pcp_raw.columns:
        if '2020' in c and ('Primary' in c or 'Physicians' in c or 'pcp' in c.lower()):
            pcp_2020_col = c
            break
    if pcp_2020_col is None:
        # fallback: any column that contains '2020'
        for c in pcp_raw.columns:
            if '2020' in c:
                pcp_2020_col = c
                break
    if pcp_2020_col is None:
        raise Exception("Could not find a PCP 2020 column in PCP file. Columns: " + ", ".join(pcp_raw.columns))

    # Population
    state_col_pop = None
    for cand in ['State_Full']:
        if cand in pop_raw.columns:
            state_col_pop = cand
            break
    if state_col_pop is None:
        raise Exception("Could not find a state column in population file. Columns: " + ", ".join(pop_raw.columns))

    pop_2020_col = None
    for cand in ['us_censusbureau_population_2020']:
        if cand in pop_raw.columns:
            pop_2020_col = cand
            break
    if pop_2020_col is None:
        # fallback: column containing '2020'
        for c in pop_raw.columns:
            if '2020' in c:
                pop_2020_col = c
                break
    if pop_2020_col is None:
        raise Exception("Could not find a 2020 population column in population file. Columns: " + ", ".join(pop_raw.columns))

    print(f"PCP state col: {state_col_pcp}, PCP 2020 col: {pcp_2020_col}")
    print(f"Population state col: {state_col_pop}, Population 2020 col: {pop_2020_col}")

    # ---------- Map state abbreviations -> full names ----------
    # This mapping includes 50 states + DC + territories (GU, PR, VI) if present
    state_lookup = {
        'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado','CT':'Connecticut','DE':'Delaware',
        'DC':'District of Columbia','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa',
        'KS':'Kansas','KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota',
        'MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
        'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon',
        'PA':'Pennsylvania','PR':'Puerto Rico','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas',
        'UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming',
        'GU':'Guam','VI':'Virgin Islands'
    }

    # Create a new column in PCP with full state names; if already full name, leave as-is
    pcp = pcp_raw.copy()
    def map_state(val):
        if pd.isna(val):
            return val
        v = str(val).strip()
        if v.upper() in state_lookup:
            return state_lookup[v.upper()]
        # if it's already a full name, return title-cased version
        return v

    pcp['State_Full'] = pcp[state_col_pcp].apply(map_state)

    # ---------- Convert PCP 2020 to numeric and aggregate by state ----------
    pcp[pcp_2020_col] = pd.to_numeric(pcp[pcp_2020_col].astype(str).str.replace(',','').str.strip(), errors='coerce')
    pcp_state = pcp.groupby('State_Full', dropna=False)[pcp_2020_col].sum(min_count=1).reset_index()
    pcp_state = pcp_state.rename(columns={pcp_2020_col: 'PCP_2020'})

    # ---------- Prepare population table ----------
    pop = pop_raw.copy()
    pop = pop.rename(columns={state_col_pop: 'State_Full'})
    # Clean numeric population column
    pop['Population_2020'] = pd.to_numeric(pop[pop_2020_col].astype(str).str.replace(',','').str.strip(), errors='coerce')

    # Merge on State_Full
    merged = pd.merge(pop[['State_Full','Population_2020']], pcp_state, on='State_Full', how='left')

    # ---------- Compute PCP per 100k ----------
    merged['PCP_2020'] = merged['PCP_2020'].fillna(0)
    merged['PCP_per_100k_2020'] = merged.apply(
        lambda r: (r['PCP_2020'] / r['Population_2020'] * 100000) if pd.notnull(r['Population_2020']) and r['Population_2020']>0 else pd.NA,
        axis=1
    )

    # ---------- Save output ----------
    out_cols = ['State_Full','Population_2020','PCP_2020','PCP_per_100k_2020']
    for c in out_cols:
        if c not in merged.columns:
            merged[c] = pd.NA

    merged = merged[out_cols]
    merged.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved merged file to: {OUTPUT_CSV}")
    print(merged.head(20).to_string(index=False))

if __name__ == "__main__":
    main()


PCP state col: State, PCP 2020 col: Primary_Care_Physicians_2020
Population state col: State_Full, Population 2020 col: us_censusbureau_population_2020
Saved merged file to: Data_Files_Cleaned/pcp_population_2020_merged.csv
          State_Full  Population_2020  PCP_2020  PCP_per_100k_2020
             Alabama          5024294    3577.0          71.194082
              Alaska           733374     743.0         101.312564
             Arizona          7157902    5269.0          73.610955
            Arkansas          3011490    2295.0          76.208123
          California         39538212   35172.0          88.956982
            Colorado          5773707    5266.0          91.206568
         Connecticut          3605912    3476.0          96.397250
            Delaware           989946     818.0          82.630770
District of Columbia           689548    1131.0         164.020489
             Florida         21538216   17215.0          79.927697
             Georgia         10713771  